In [ ]:
# mount drive into colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/Bazaar/prediction'

/content/drive/MyDrive/Bazaar/prediction


# Imports

In [ ]:
%%capture
!pip install pytorch-lightning

In [ ]:
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split

import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers

# Load Data

In [ ]:
df = pd.read_csv('train.csv')
df['X'] = df['historical_games'].str.split(' ').apply(lambda x: [int(i) for i in x])
df['Y'] = df['next_game'].apply(lambda x: int(x))
df.head()

,id,historical_games,next_game,X,Y
0,2,3 12 262 6094 283 50 1070 233,131,"[3, 12, 262, 6094, 283, 50, 1070, 233]",131
1,4,294 241 1 150 12,101,"[294, 241, 1, 150, 12]",101
2,7,85 139 144 57 2013,330,"[85, 139, 144, 57, 2013]",330
3,10,7 114 10 5 31 6504,19,"[7, 114, 10, 5, 31, 6504]",19
4,18,5 221 3 712 159 4 810 94 746 6170 136 17 1160 ...,247,"[5, 221, 3, 712, 159, 4, 810, 94, 746, 6170, 1...",247


In [ ]:
df_test = pd.read_csv('test.csv')
df_test['X'] = df_test['historical_games'].str.split(' ').apply(lambda x: [int(i) for i in x])
df_test.head()

,id,historical_games,X
0,5,46 37 3 118 3135 683 213 298,"[46, 37, 3, 118, 3135, 683, 213, 298]"
1,13,1528 537 287 2023 93 417 4442 1313 245 6234 94...,"[1528, 537, 287, 2023, 93, 417, 4442, 1313, 24..."
2,20,207 5 19 332 111,"[207, 5, 19, 332, 111]"
3,25,1412 18 18 3050 728 5 616,"[1412, 18, 18, 3050, 728, 5, 616]"
4,32,16 64 1 294 449,"[16, 64, 1, 294, 449]"


In [ ]:
# Combine all data to compute vocab size
all_game_ids = (df['X'] + df['Y'].apply(lambda x: [x])).tolist()
all_game_ids += df_test['X'].values.tolist()
all_game_ids = [id for ids in all_game_ids for id in ids]
unique_game_ids, counts = np.unique(all_game_ids, return_counts=True)
print(f'Number of game ids: {len(unique_game_ids)}')

Number of game ids: 7804


In [ ]:
index_to_game_id = {index: game_id for index, game_id in enumerate(unique_game_ids)}
game_id_to_index = {game_id: index for index, game_id in enumerate(unique_game_ids)}
game_id_indexes = [game_id_to_index[w] for w in unique_game_ids]

In [ ]:
df['X_encoded'] = df['X'].apply(lambda x: [game_id_to_index[id] for id in x])
df['Y_encoded'] = df['Y'].apply(lambda x: game_id_to_index[x])
df.head()

,id,historical_games,next_game,X,Y,X_encoded,Y_encoded
0,2,3 12 262 6094 283 50 1070 233,131,"[3, 12, 262, 6094, 283, 50, 1070, 233]",131,"[2, 11, 261, 6090, 282, 49, 1069, 232]",130
1,4,294 241 1 150 12,101,"[294, 241, 1, 150, 12]",101,"[293, 240, 0, 149, 11]",100
2,7,85 139 144 57 2013,330,"[85, 139, 144, 57, 2013]",330,"[84, 138, 143, 56, 2012]",329
3,10,7 114 10 5 31 6504,19,"[7, 114, 10, 5, 31, 6504]",19,"[6, 113, 9, 4, 30, 6500]",18
4,18,5 221 3 712 159 4 810 94 746 6170 136 17 1160 ...,247,"[5, 221, 3, 712, 159, 4, 810, 94, 746, 6170, 1...",247,"[4, 220, 2, 711, 158, 3, 809, 93, 745, 6166, 1...",246


In [ ]:
df_test['X_encoded'] = df_test['X'].apply(lambda x: [game_id_to_index[id] for id in x])
df_test.head()

,id,historical_games,X,X_encoded
0,5,46 37 3 118 3135 683 213 298,"[46, 37, 3, 118, 3135, 683, 213, 298]","[45, 36, 2, 117, 3133, 682, 212, 297]"
1,13,1528 537 287 2023 93 417 4442 1313 245 6234 94...,"[1528, 537, 287, 2023, 93, 417, 4442, 1313, 24...","[1527, 536, 286, 2022, 92, 416, 4440, 1312, 24..."
2,20,207 5 19 332 111,"[207, 5, 19, 332, 111]","[206, 4, 18, 331, 110]"
3,25,1412 18 18 3050 728 5 616,"[1412, 18, 18, 3050, 728, 5, 616]","[1411, 17, 17, 3048, 727, 4, 615]"
4,32,16 64 1 294 449,"[16, 64, 1, 294, 449]","[15, 63, 0, 293, 448]"


# Dataset Loader

In [ ]:
class CustomTextDataset(Dataset):
  def __init__(self, X, y=None):
    if y is not None:
      self.data = list(zip(X,y))
      self.data = sorted(self.data, key=lambda x: len(x[0]))
    else:
      self.data = X
      
  def __len__(self):
      return len(self.data)

  def __getitem__(self, idx):
      return self.data[idx]

In [ ]:
def collate_text(batch):
  data = [item[0] for item in batch]
  target = [item[1] for item in batch]
  max_seq_len = max([len(x) for x in data])
  data = [np.pad(p, (0, max_seq_len - len(p)), 'constant') for p in data]
  data = torch.IntTensor(data)
  target = torch.LongTensor(target)
  return [data, target]

In [ ]:
dataset = CustomTextDataset(df['X_encoded'].values.tolist(), df['Y_encoded'].values.tolist())

In [ ]:
dataset.__getitem__(5)

([6, 8, 26, 12, 9], 59)

# BiLSTM Model

In [ ]:
class BiLSTM(pl.LightningModule):
  def __init__(self, vocab_size):
    super().__init__()
    
    # Configurations
    self.hidden_size = 2
    self.linear_dim = 64
    drp = 0.5
    embed_size = 50
    
    # Layers
    self.embedding = nn.Embedding(vocab_size, embed_size)
    self.lstm = nn.LSTM(embed_size, self.hidden_size, bidirectional=True, batch_first=True)
    self.linear = nn.Linear(self.hidden_size*4 , self.linear_dim)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(drp)
    self.out = nn.Linear(self.linear_dim, vocab_size)

  def forward(self, x):
    h_embedding = self.embedding(x)
    h_embedding = torch.squeeze(torch.unsqueeze(h_embedding, 0))

    h_lstm, _ = self.lstm(h_embedding)
    avg_pool = torch.mean(h_lstm, 1)
    max_pool, _ = torch.max(h_lstm, 1)
    conc = torch.cat(( avg_pool, max_pool), 1)
    conc = self.relu(self.linear(conc))
    conc = self.dropout(conc)
    out = self.out(conc)
    return out

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=5e-4)
    return optimizer

  def training_step(self, train_batch, batch_idx):
    x, y = train_batch 
    y_output = self.forward(x)
    loss = F.cross_entropy(y_output, y)
    self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    return loss

  def validation_step(self, val_batch, batch_idx):
    x, y = val_batch
    y_output = self.forward(x)
    loss = F.cross_entropy(y_output, y)
    self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

# Training

In [ ]:
# Configs
VAL_RATIO = 0.1
BATCH_SIZE = 512
MAX_EPOCHS = 200

In [ ]:
# model
model = BiLSTM(vocab_size=len(unique_game_ids))

# data
val_size = round(VAL_RATIO * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset,batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_text)
val_dataloader = DataLoader(val_dataset,batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_text)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Train dataset size: 27529
Validation dataset size: 3059


In [ ]:
# training
tb_logger = pl_loggers.TensorBoardLogger('logs/')

trainer = pl.Trainer(gpus=1, max_epochs=MAX_EPOCHS, logger=tb_logger)
trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | embedding | Embedding | 390 K 
1 | lstm      | LSTM      | 864   
2 | linear    | Linear    | 576   
3 | relu      | ReLU      | 0     
4 | dropout   | Dropout   | 0     
5 | out       | Linear    | 507 K 
----------------------------------------
898 K     Trainable params
0         Non-trainable params
898 K     Total params
3.596     Total estimated model params size (MB)


In [ ]:
#%reload_ext tensorboard
#%tensorboard --logdir logs

# Prediction

In [ ]:
PREDICTION_STEPS = 20
test_predictions = []

for game_seq in df_test['X_encoded'].values.tolist():
  model_input = game_seq.copy()
  predictions = []
  
  for step in range(PREDICTION_STEPS):
    raw_prediction = model.forward(torch.IntTensor([model_input, model_input]))[0]
    next_game_id = torch.argmax(raw_prediction)
    model_input.append(next_game_id.item())
    predictions.append(index_to_game_id[ next_game_id.item() ])
  
  test_predictions.append(predictions)

In [ ]:
def unique_everseen_limit(iterable, limit=5):
  seen = set()
  seen_add = seen.add
  for element in iterable:
    if element not in seen:
      seen_add(element)
      yield element
    if len(seen) == limit:
      break

In [ ]:
# seperate first 5 unique predictions
first_unique_preds = []
for item in test_predictions:
  first_unique_preds.append(list(unique_everseen_limit(item)))

In [ ]:
df_test['next_games'] = [" ".join(str(id) for id in predictions) for predictions in first_unique_preds]
df_test.head()

,id,historical_games,X,X_encoded,next_games
0,5,46 37 3 118 3135 683 213 298,"[46, 37, 3, 118, 3135, 683, 213, 298]","[45, 36, 2, 117, 3133, 682, 212, 297]",122 92 194 734 436
1,13,1528 537 287 2023 93 417 4442 1313 245 6234 94...,"[1528, 537, 287, 2023, 93, 417, 4442, 1313, 24...","[1527, 536, 286, 2022, 92, 416, 4440, 1312, 24...",80 49 524 110 2
2,20,207 5 19 332 111,"[207, 5, 19, 332, 111]","[206, 4, 18, 331, 110]",49 542 192 80 110
3,25,1412 18 18 3050 728 5 616,"[1412, 18, 18, 3050, 728, 5, 616]","[1411, 17, 17, 3048, 727, 4, 615]",15 49 23 14 6
4,32,16 64 1 294 449,"[16, 64, 1, 294, 449]","[15, 63, 0, 293, 448]",49 734 524 533 78


In [ ]:
df_test[['id', 'next_games']].to_csv('prediction.csv', index=False)